# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [21]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2024-06-01	A Lack of Mass-gap Compact Object Binaries in APOGEE	Research Notes of the American Astronomical Society  Meir Schochet et al 2024 Res. Notes AAS 8 166 3.	Depending principally on mass, the compact object remnant left behind after a star’s life may be a white dwarf (WD), neutron star (NS), or black hole (BH). While we have large samples of each of these remnants, we lack knowledge of the exact conditions separating these outcomes. The boundary between low-mass BHs and massive NSs is particularly poorly understood, as few objects between 2-5 M⊙ are known. To probe this regime, we search the APOGEE DR17 dataset of 657,000 unique stars for binary systems with one stellar component that exhibit large radial velocity shifts over multiple observations. We identify 4751 likely binary systems, and estimate a minimum mass for each system’s “invisible companion” under the assumption of tidal synchronization. Two syste

## Import pandas

We are using the very handy pandas library for dataframes.

In [29]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [30]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-06-01,A Lack of Mass-gap Compact Object Binaries in ...,Research Notes of the American Astronomical So...,"Depending principally on mass, the compact obj...",Meir Schochet et al 2024 Res. Notes AAS 8 166,a-lack-of-mass-gap-COs-in-APOGEE,https://m-schochet.github.io/files/10.3847/251...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [31]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [32]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [33]:
!ls ../_publications/

2024-06-01-a-lack-of-mass-gap-COs-in-APOGEE.md


In [34]:
!ls ../_publications/2024-06-01-a-lack-of-mass-gap-COs-in-APOGEE.md

../_publications/2024-06-01-a-lack-of-mass-gap-COs-in-APOGEE.md


In [36]:
!cat ../_publications/2024-06-01-a-lack-of-mass-gap-COs-in-APOGEE.md

---
title: "A Lack of Mass-gap Compact Object Binaries in APOGEE"
collection: publications
permalink: /publication/2024-06-01-a-lack-of-mass-gap-COs-in-APOGEE
excerpt: 'Depending principally on mass, the compact object remnant left behind after a star’s life may be a white dwarf (WD), neutron star (NS), or black hole (BH). While we have large samples of each of these remnants, we lack knowledge of the exact conditions separating these outcomes. The boundary between low-mass BHs and massive NSs is particularly poorly understood, as few objects between 2-5 M⊙ are known. To probe this regime, we search the APOGEE DR17 dataset of 657,000 unique stars for binary systems with one stellar component that exhibit large radial velocity shifts over multiple observations. We identify 4751 likely binary systems, and estimate a minimum mass for each system’s “invisible companion” under the assumption of tidal synchronization. Two systems have companion masses ≳ 2 M⊙, although we conclude that neithe